In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip

In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.loc[:,"toxic"].unique()

In [ ]:
categories = list(df.iloc[:,2:].columns)
categories

In [ ]:
df1 = df.iloc[:,2:]
counts=[]
for i in categories:
    counts.append((i, df1[i].sum()))

data = pd.DataFrame(counts, columns=["category", "no_of_comments"])
data   
    

In [ ]:
data.plot(x="category", y="no_of_comments", kind="bar")

In [ ]:
rowsums = df.iloc[:, 2:].sum(axis=1)
import matplotlib.pyplot as plt
plt.bar(rowsums.value_counts().reset_index()["index"],rowsums.value_counts().reset_index()[0])

In [ ]:
rowsums.value_counts()

In [ ]:
df.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
m1 = CountVectorizer()
x = m1.fit_transform(df["comment_text"])

In [ ]:
len(m1.get_feature_names())

In [ ]:
m1.get_feature_names()

In [ ]:
df["comment_text"] = df["comment_text"].str.replace("\d","")

In [ ]:
import re
def cleaning(text):
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"\'ll","will", text)
    text = re.sub(r"\'m","am", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    return text

In [ ]:
df["comment_text"] = df["comment_text"].apply(cleaning)

In [ ]:
df.head()

In [ ]:
import nltk
from nltk.corpus import stopwords
list1 = stopwords.words("english")
list1.extend(['aa',
 'aaa',
 'aaaa',
 'aaaaa',
 'aaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaany',
 'aaaaaaaaaah',
 'aaaaaaaaaahhhhhhhhhhhhhh',
 'aaaaaaaaadm',
 'aaaaaaaaaq',
 'aaaaaaaacfo',
 'aaaaaaaaczy',
 'aaaaaaaahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh',
 'aaaaaaaari',
 'aaaaaaaayui',
 'aaaaaaahhhhhhhhhhhhhhhhhhhhhhhh',
 'aaaaaaw',
 'aaaaah',
 'aaaah',
 'aaaannnnyyyywwwwhhhheeeerrrreeee',
 'aaaawwww',
 'aaaboyz',
 'aaages',
 'aaaghh',
 'aaah',
 'aaahhh',
 'aaahs',
 'aaai',
 'aaajade',
 'aaand',
 'aaarrrgggh',
 'aaaww',
 'aab',
 'aaba',])

In [ ]:
list1

In [ ]:
def stop(text):
    text = [i for i in text.split() if i not in list1]
    return " ".join(text)
df["comment_text"] = df["comment_text"].apply(stop)

In [ ]:
df = df.iloc[:10000,:]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
m2 = CountVectorizer(ngram_range=(1,2))
ngram = m2.fit_transform(df["comment_text"].values)
ngram

In [ ]:
print(m2.get_feature_names())

In [ ]:
count_values = ngram.toarray().sum(axis=0)

In [ ]:
vocab = m2.vocabulary_
len(vocab.keys())

In [ ]:
# count_values[85423]
# the=85423

In [ ]:
d2 = pd.DataFrame(sorted([(count_values[j], i) for i, j in vocab.items()],reverse=True)).rename(columns={0:"Frequency",1:"Unigram/Bigram"})

In [ ]:
list(d2[d2["Frequency"]==1]["Unigram/Bigram"])

In [ ]:
X = df["comment_text"]
Y = df.iloc[:, 2:]

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest, ytrain, ytest = train_test_split(X,Y, test_size=0.2, random_state=41)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(max_features=1000,
               analyzer="word",
               stop_words="english", norm="l2")

In [ ]:
xtrain_tfidf = model.fit_transform(xtrain)
xtest_tfidf = model.transform(xtest)

In [ ]:
xtrain_tfidf

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
a12 = RandomForestClassifier()
model1 = MultiOutputClassifier(estimator= a12)
model1.fit(xtrain_tfidf, ytrain)

In [ ]:
list3 = Y.columns

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred = model1.predict(xtrain_tfidf)
print("Training Results")
for i in range(len(list3)):
    acc1 = accuracy_score(ytrain.iloc[:, i], pred[:,i])
    print(list3[i]+ "==>", acc1)


In [ ]:
pred1 = model1.predict(xtest_tfidf)
print("Testing Results")
for i in range(len(list3)):
    acc2 = accuracy_score(ytest.iloc[:, i], pred1[:,i])
    print(list3[i]+ "==>", acc2)


In [ ]:
x_tfidf= model.transform(X)
predicted = model1.predict(x_tfidf)


print(classification_report(Y, predicted, target_names=list3))